In [2]:
from PIL import Image
from torch import nn 
from torchvision import models, transforms
import torch.onnx 
import requests
import numpy as np 

In [3]:
BATCH_SIZE=64

In [4]:
sample_img = Image.open(
    requests.get(
        "http://assets.myntassets.com/v1/images/style/properties/7a5b82d1372a7a5c6de67ae7a314fd91_images.jpg", 
        stream=True)\
    .raw)

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
x = torch.randn(1, 3, 224, 224)

In [7]:
base_model  = models.vgg16(pretrained=True)

/home/james/anaconda3/envs/torch_cv_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/james/anaconda3/envs/torch_cv_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
model = base_model.to(device)

In [9]:
base_model.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [10]:
trans = transforms.Compose([
    transforms.Resize((244)),
    transforms.ToTensor(),
])

In [11]:
trans

Compose(
    Resize(size=244, interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
)

In [12]:
sample_input = trans(sample_img)

In [13]:
sample_input.shape

torch.Size([3, 325, 244])

In [14]:
from torch.utils.data import DataLoader 

In [15]:
torch.save(model, "models/vgg16.torch")

In [16]:
cuda_input = sample_input.to(device)

In [17]:
cuda_input.shape

torch.Size([3, 325, 244])

In [18]:
cuda_input_final = cuda_input.unsqueeze(0)

In [19]:
model(cuda_input_final)

tensor([[ 4.3843e-01, -1.1409e+00, -1.8003e+00, -1.9813e+00, -6.2103e-01,
          1.4748e-01, -1.2773e+00, -1.4748e+00, -1.8193e+00, -3.3324e+00,
         -2.6636e+00, -3.0319e+00, -2.6548e+00, -2.7222e+00, -1.4038e+00,
         -2.6484e+00, -2.4838e+00, -1.9218e+00, -2.9215e+00, -1.9168e+00,
         -2.8512e+00, -1.7300e+00, -3.0235e+00, -3.1577e+00, -2.0754e+00,
         -2.2516e+00, -1.5841e+00, -1.1413e+00, -7.1804e-01, -1.9621e+00,
         -2.3418e+00, -2.7144e+00, -1.7334e+00, -7.4089e-01, -2.2293e-01,
         -1.9706e+00, -6.5241e-01, -1.3567e+00, -6.9290e-01, -2.1339e+00,
         -2.0594e+00, -2.1832e+00, -2.9453e+00, -1.9408e+00, -1.8848e+00,
         -1.1064e+00, -1.3633e+00, -1.1942e+00, -1.9714e+00, -2.2032e+00,
          2.2160e-02, -1.1443e-01, -2.1405e-01, -6.3977e-01, -1.4040e+00,
         -7.7976e-01, -2.2681e+00, -2.2844e+00, -1.9249e+00, -4.6367e-01,
         -1.0582e+00, -1.4732e+00, -1.0486e+00, -7.1847e-01, -3.5748e-01,
         -1.0640e+00, -1.1700e+00, -7.

In [20]:
244*244*3

178608

In [21]:
64*64*3

12288

In [22]:
import math 
math.sqrt(4096)

64.0

In [23]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [24]:
x_cuda = x.to(device)

In [25]:
x_cuda.shape

torch.Size([1, 3, 224, 224])

In [26]:
model(x_cuda)

tensor([[ 4.9399e-01,  3.6448e+00, -1.4252e+00, -7.7867e-01,  8.7936e-01,
          3.4538e+00,  3.1414e+00,  7.9657e-01,  6.6070e-01, -4.5570e-01,
         -8.8170e-02,  2.2054e+00,  7.0811e-01,  3.3343e-01,  5.8564e-01,
          1.3328e+00, -2.3455e+00, -1.5847e+00,  1.3300e+00, -1.0676e+00,
         -1.2350e+00, -1.8543e+00, -1.9166e-01, -2.4735e-01, -1.3031e+00,
         -1.7595e+00,  1.6076e-01, -1.1982e-01, -1.4511e+00,  4.6960e-01,
         -1.5526e+00, -1.3142e+00, -2.5233e+00, -2.7358e-01, -2.9435e-03,
         -9.7851e-01, -6.6806e-01, -1.7022e+00, -1.1097e+00, -1.7161e+00,
          1.6548e-01, -2.2839e+00, -9.1546e-01, -8.3508e-01, -2.2171e+00,
         -4.4755e-01,  6.5733e-01, -1.1186e+00, -4.4587e-01, -1.4346e+00,
         -1.8483e-01, -2.4476e+00, -9.5768e-01, -1.6762e+00, -1.8545e+00,
         -1.0092e+00, -2.9791e+00, -3.0366e+00,  1.4077e-01, -1.8119e+00,
         -8.3329e-01, -2.9841e+00, -2.8301e+00, -1.0909e+00, -8.0770e-01,
          1.7635e+00, -1.6432e+00, -1.

### Export Torchscript

In [27]:
example = open("models/vgg_torchscript.jit", "wb")
script = torch.jit.script(model, cuda_input_final)

/home/james/anaconda3/envs/torch_cv_env/lib/python3.11/site-packages/torch/jit/_script.py:1277: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


In [28]:
script.save("models/vgg_torchscript.jit")

In [29]:
script_vgg = torch.jit.load("models/vgg_torchscript.jit")
script_vgg.eval()

RecursiveScriptModule(
  original_name=VGG
  (features): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=ReLU)
    (2): RecursiveScriptModule(original_name=Conv2d)
    (3): RecursiveScriptModule(original_name=ReLU)
    (4): RecursiveScriptModule(original_name=MaxPool2d)
    (5): RecursiveScriptModule(original_name=Conv2d)
    (6): RecursiveScriptModule(original_name=ReLU)
    (7): RecursiveScriptModule(original_name=Conv2d)
    (8): RecursiveScriptModule(original_name=ReLU)
    (9): RecursiveScriptModule(original_name=MaxPool2d)
    (10): RecursiveScriptModule(original_name=Conv2d)
    (11): RecursiveScriptModule(original_name=ReLU)
    (12): RecursiveScriptModule(original_name=Conv2d)
    (13): RecursiveScriptModule(original_name=ReLU)
    (14): RecursiveScriptModule(original_name=Conv2d)
    (15): RecursiveScriptModule(original_name=ReLU)
    (16): RecursiveScriptModule(original_name

### Export Onnx

In [30]:
path = "models/vgg_torch.onnx"
torch.onnx.export(
    script_vgg, 
    cuda_input_final, 
    path)

/home/james/anaconda3/envs/torch_cv_env/lib/python3.11/site-packages/torch/onnx/utils.py:847: UserWarning: no signature found for <torch.ScriptMethod object at 0x7f16f893bbf0>, skipping _decide_input_format
  warnings.warn(f"{e}, skipping _decide_input_format")


SymbolicValueError: Unsupported: ONNX export of operator adaptive_avg_pool2d, output size that are not factor of input size. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues  [Caused by the value '34 defined in (%34 : Long(2, strides=[1], device=cpu) = onnx::Constant[value= 7  7 [ CPULongType{2} ]]()
)' (type 'Tensor') in the TorchScript graph. The containing node has kind 'onnx::Constant'.] 

    Inputs:
        Empty
    Outputs:
        #0: 34 defined in (%34 : Long(2, strides=[1], device=cpu) = onnx::Constant[value= 7  7 [ CPULongType{2} ]]()
    )  (type 'Tensor')